<a href="https://colab.research.google.com/github/abhipsitabose/AIP/blob/main/Assignment_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
upload = files.upload()

Saving olist_customers_dataset.csv to olist_customers_dataset.csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset.csv
Saving olist_order_reviews_dataset.csv to olist_order_reviews_dataset.csv
Saving olist_orders_dataset.csv to olist_orders_dataset.csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset.csv


In [3]:
order_reviews_df = pd.read_csv('olist_order_reviews_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
orders_df = pd.read_csv('olist_orders_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')
sellers_df = pd.read_csv('olist_sellers_dataset.csv')

In [4]:
# Merge order_reviews with orders (use order_id)
joined_df = pd.merge(order_reviews_df, orders_df, on='order_id', how='left')

# Merge with customers to get customer demographics (use customer_id)
joined_df = pd.merge(joined_df, customers_df, on='customer_id', how='left')

# Merge with order items (use order_id)
joined_df = pd.merge(joined_df, order_items_df, on='order_id', how='left')

# Merge with sellers (use seller_id)
final_df = pd.merge(joined_df, sellers_df, on='seller_id', how='left')

In [11]:
# Information about the joined dataset (Can remove this code in the final version, this to check intermediate stage)
print(final_df.info())
print(final_df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 99857 entries, 0 to 102776
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   review_id                      99857 non-null  object 
 1   order_id                       99857 non-null  object 
 2   review_score                   99857 non-null  int64  
 3   review_comment_title           12045 non-null  object 
 4   review_comment_message         42323 non-null  object 
 5   review_creation_date           99857 non-null  object 
 6   review_answer_timestamp        99857 non-null  object 
 7   customer_id                    99857 non-null  object 
 8   order_status                   99857 non-null  object 
 9   order_purchase_timestamp       99857 non-null  object 
 10  order_approved_at              99857 non-null  object 
 11  order_delivered_carrier_date   99857 non-null  object 
 12  order_delivered_customer_date  99857 non-null  obj

In [6]:
#Drop rows where order_delivered_customer_date or order_delivered_carrier_date are missing
final_df = final_df.dropna(subset=['order_delivered_customer_date', 'order_delivered_carrier_date'])


In [10]:
# Drop rows where 'seller_id' is NaN or empty
final_df = final_df[
    final_df['order_approved_at'].notna() & final_df['seller_id'].notna() &
    (final_df['order_approved_at'] != '') & (final_df['seller_id'] != '')
]


In [15]:
#Remove duplicate records based on review ID
# Sort by 'review_id'
final_df.sort_values(by=['review_id'], inplace=True)

# Drop duplicates and keep the first instance
final_df = final_df[~final_df.duplicated(subset=['review_id'], keep='first')]


In [16]:
# Find rows with duplicate 'order_id' but same or different 'review_score' and keep the most recent one
# Sort by 'order_id' and 'review_creation_date'
final_df.sort_values(by=['order_id', 'review_creation_date'], inplace=True)

# Keep only the latest review for each 'order_id'
final_df = final_df.drop_duplicates(subset=['order_id'], keep='last')

In [17]:
# Count the number of duplicates in each column (will drop this code for final version)
for column in final_df.columns:
    duplicates_count = final_df[column].duplicated().sum()
    print(f"Number of duplicates in column '{column}': {duplicates_count}")


Number of duplicates in column 'review_id': 0
Number of duplicates in column 'order_id': 0
Number of duplicates in column 'review_score': 86478
Number of duplicates in column 'review_comment_title': 82423
Number of duplicates in column 'review_comment_message': 54890
Number of duplicates in column 'review_creation_date': 85857
Number of duplicates in column 'review_answer_timestamp': 7532
Number of duplicates in column 'customer_id': 0
Number of duplicates in column 'order_status': 86481
Number of duplicates in column 'order_purchase_timestamp': 289
Number of duplicates in column 'order_approved_at': 6699
Number of duplicates in column 'order_delivered_carrier_date': 13723
Number of duplicates in column 'order_delivered_customer_date': 633
Number of duplicates in column 'order_estimated_delivery_date': 86040
Number of duplicates in column 'customer_unique_id': 2213
Number of duplicates in column 'customer_zip_code_prefix': 71942
Number of duplicates in column 'customer_city': 82523
Num

In [18]:
#drop columns not required
#final_df= final_df.drop(['review_id','order_id','customer_id''review_comment_title','review_comment_message','freight_value','customer_zip_code_prefix','customer_city','customer_state','order_item_id','product_id','price','seller_zip_code_prefix','seller_city','seller_state'], axis=1)
final_df= final_df.drop(['review_comment_title','review_comment_message','freight_value','customer_zip_code_prefix','customer_state','order_item_id','product_id','price','seller_zip_code_prefix','seller_state'], axis=1)
final_df.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_city,seller_id,shipping_limit_date,seller_city
0,97ca439bc427b48bc1cd7177abe71365,00010242fe8c5a6d1ba2dd792cb16214,5,21/09/2017 00:00,22/09/2017 10:57,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,campos dos goytacazes,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,volta redonda
1,7b07bacd811c4117b742569b04ce3580,00018f77f2f0320c557190d7a144bdd3,4,13/05/2017 00:00,15/05/2017 11:34,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,santa fe do sul,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,sao paulo
2,0c5b33dea94867d1ac402749e5438e8b,000229ec398224ef6ca0657da4fc703e,5,23/01/2018 00:00,23/01/2018 16:06,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,para de minas,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,borda da mata
3,f4028d019cb58564807486a6aaf33817,00024acbcdf0a6daa1e931b038114c75,4,15/08/2018 00:00,15/08/2018 16:39,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,atibaia,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,franca
4,940144190dcba6351888cafa43f3a3a5,00042b26cf59d7ce69dfabb4e55b4fd9,5,02/03/2017 00:00,03/03/2017 10:54,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,varzea paulista,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,loanda


In [ ]:
from google.colab import files

# Save the joined DataFrame to CSV
final_df.to_csv('final_df.csv', index=False)

# Download the file to your local machine
files.download('final_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86483 entries, 0 to 102776
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   review_id                      86483 non-null  object
 1   order_id                       86483 non-null  object
 2   review_score                   86483 non-null  int64 
 3   review_creation_date           86483 non-null  object
 4   review_answer_timestamp        86483 non-null  object
 5   customer_id                    86483 non-null  object
 6   order_status                   86483 non-null  object
 7   order_purchase_timestamp       86483 non-null  object
 8   order_approved_at              86483 non-null  object
 9   order_delivered_carrier_date   86483 non-null  object
 10  order_delivered_customer_date  86483 non-null  object
 11  order_estimated_delivery_date  86483 non-null  object
 12  customer_unique_id             86483 non-null  object
 13  custo

In [20]:
final_df.isnull().sum()

,0
review_id,0
order_id,0
review_score,0
review_creation_date,0
review_answer_timestamp,0
customer_id,0
order_status,0
order_purchase_timestamp,0
order_approved_at,0
order_delivered_carrier_date,0


In [ ]:
#FEATURE -1 LATE DELIVERY

#Are orders being delivered late
final_df['order_delivered_customer_date'] = pd.to_datetime(final_df['order_delivered_customer_date'])
final_df['order_estimated_delivery_date'] = pd.to_datetime(final_df['order_estimated_delivery_date'])
final_df['late_delivery'] = (final_df['order_delivered_customer_date'] > final_df['order_estimated_delivery_date']).astype(int)

In [22]:
#FEATURE 2 - TIME TO APPROVE ORDER
# Calculate the time taken to approve order in days
final_df['order_purchase_timestamp'] = pd.to_datetime(final_df['order_purchase_timestamp'])
final_df['order_approved_at'] = pd.to_datetime(final_df['order_approved_at'])
final_df['processing_time_approve'] = (final_df['order_approved_at'] - final_df['order_purchase_timestamp']).dt.days

In [24]:
#FEATURE 2 - TIME TO PROCESS ORDER
# Calculate the seller processing time in days (The time taken by the seller to prepare and ship the order
final_df['order_delivered_carrier_date'] = pd.to_datetime(final_df['order_delivered_carrier_date'])
final_df['processing_time'] = (final_df['order_delivered_carrier_date'] - final_df['order_approved_at']).dt.days


In [25]:
# FEATURE - 3 TIME TO SHIP

#Calculate Processing Time (Time to Ship). Processing time is the difference between the order_approved_at (when payment was approved) and order_delivered_carrier_date (when the order was handed to the carrier).
# Ensure dates are in datetime format
final_df['order_approved_at'] = pd.to_datetime(final_df['order_approved_at'])
final_df['order_delivered_carrier_date'] = pd.to_datetime(final_df['order_delivered_carrier_date'])

# Calculate processing time in days
final_df['processing_time'] = (final_df['order_delivered_carrier_date'] - final_df['order_approved_at']).dt.days

#Evaluate if Processing Time is Within an Acceptable Range
# Define threshold for acceptable processing time
processing_threshold = 2

# Boolean column: True if processing time is within threshold
final_df['processing_within_threshold'] = final_df['processing_time'] <= processing_threshold



In [26]:
# FEATURE - 3 LATE SHIPMENTS
#We can determine whether sellers are adhering to the shipping_limit_date by check if the order was shipped late
final_df['late_shipment'] = final_df['order_delivered_carrier_date'] > final_df['shipping_limit_date']